In [1]:
#create new file pipeline_build.ipynb

import azureml.core
%pip install azureml-pipeline-steps
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

Note: you may need to restart the kernel to use updated packages.


In [2]:
datastore = Datastore.get(ws, "demostore")

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
compute_name = "firstmohit"
vm_size = "Standard_E4s_v3"
compute_target = ws.compute_targets[compute_name]

In [4]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment

aml_run_config = RunConfiguration()
# compute target as defined in "Azure Machine Learning compute" section above
aml_run_config.target = compute_target

USE_CURATED_ENV = True
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn'], 
        pip_packages=['azureml-sdk', 'azureml-dataset-runtime[fuse,pandas]'], 
        pin_sdk_version=False)

In [5]:

from azureml.core import Dataset
from azureml.data import OutputFileDatasetConfig

from azureml.pipeline.steps import PythonScriptStep
#dataprep_source_dir = "./dataprep_src"
entry_point = "python_script.py"

In [6]:
py_script_run_step = PythonScriptStep(
    script_name = entry_point,
    compute_target = compute_target,
    runconfig = aml_run_config,
    allow_reuse = False
)

In [7]:
pipeline_steps = [py_script_run_step]

In [8]:
from azureml.pipeline.core import Pipeline

# build the pipeline
pipeline1 = Pipeline(workspace =ws, steps= [pipeline_steps])

In [9]:
from azureml.core import Experiment

#Submit the pipeline run
pipeline_run1 = Experiment(ws, 'first_trial-of-custom_pipeline').submit(pipeline1)
pipeline_run1.wait_for_completion(show_output = True)


Created step python_script.py [d142debc][0696e352-f808-472b-8268-454260e212f7], (This step will run and generate new outputs)
Submitted PipelineRun 204bf167-8ce8-4ed8-984a-54f3b12e8922
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/204bf167-8ce8-4ed8-984a-54f3b12e8922?wsid=/subscriptions/ce687e80-2716-4878-ba12-099e3e2e73b7/resourcegroups/machinelearning/workspaces/azuremlmohit&tid=7158f100-9188-4d39-a152-f7780b9cc57e
PipelineRunId: 204bf167-8ce8-4ed8-984a-54f3b12e8922
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/204bf167-8ce8-4ed8-984a-54f3b12e8922?wsid=/subscriptions/ce687e80-2716-4878-ba12-099e3e2e73b7/resourcegroups/machinelearning/workspaces/azuremlmohit&tid=7158f100-9188-4d39-a152-f7780b9cc57e
PipelineRun Status: Running


StepRunId: a46d0234-7052-4958-882d-27f8aae1bccf
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/a46d0234-7052-4958-882d-27f8aae1bccf?wsid=/subscriptions/ce687e80-2716-4878-ba12-099e3e2e73b7/resourcegroups/m

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process 'python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"python_script.py\", line 44, in <module>\n    print(\"count record from the file \"+ str(training_data.shape[0]))\nAttributeError: 'TabularDataset' object has no attribute 'shape'\n\n",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z",
    "componentName": "CommonRuntime"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Execution failed. User process 'python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\\n